In [151]:
# Author: Meysam Aghighi
# Contact: meysam.aghighi@gmail.com
# Date: 2024-11-08
# Description: A script to automate desk booking in flowscape

import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime, timedelta

In [152]:
today = datetime.today()

building = "SEKI10 Kista"
floor = "Floor 8"
desk = "326A"
dates = [(today + timedelta(days=i)).day for i in range(16)]
# dates = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]

# Load cookies from the saved JSON file
with open("flowscape_cookies.json", "r") as f:
    cookies = json.load(f)

# Open Chrome
chrome_options = Options() # this is important for cookies to work
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()

# Go to webpage
driver.get("https://ericsson.flowscape.se/webapp/")

# Add each cookie to the session
for cookie in cookies:    
#     cookie.pop('sameSite', None)
    driver.add_cookie(cookie)
    
driver.refresh()

# Select building
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f"//button[text()='{building}']"))
)
button.click()
# Select floor
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f"//button[text()='{floor}']"))
)
button.click()

print(f"searching the desk {desk}...")
# Search the desk
search_field = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Search']"))
)
search_field.send_keys(desk)
time.sleep(1)
print("click on the first search...")
# Open the first search (desk)
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'sc-qZtCU cJnHnZ')]"))
)
button.click()

div_element = WebDriverWait(driver, 10).until(
EC.presence_of_element_located((By.CSS_SELECTOR, "div.sc-pBxWu.gglDmq"))
)

# Book desk in all dates
for date in dates:
    print(f"selecting the date {date}...")
    # Select date
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//p[contains(@class, 'MuiTypography-root') and text()='{date}']"))
    )
    element.click()
    time.sleep(1)
    try:
        # Try to find the <span> with the desk_number text inside the <div>
        desk_number_span = div_element.find_element(By.TAG_NAME, "span")

        # Check if the desk number matches the expected value
        if desk_number_span.text == desk:
            print(f"Desk number {desk} is found. Booking...")
            # Book the desk
            try:
                book_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, "button.sc-oVpqz.lpruGk"))
                )
                book_button.click()
            except TimeoutException:
                print(f"Desk number {desk} is not able to be booked on date {date}!")
        else:
            print(f"Desk number found: {desk_number_span.text}. No action performed. Check the code.")

    except NoSuchElementException:
        print("No <span> with desk_number found inside the <div>. Skipping action.")

searching the desk 326A...
click on the first search...
selecting the date 9...
Desk number 326A is found. Booking...
selecting the date 10...
Desk number 326A is found. Booking...
selecting the date 11...
Desk number 326A is found. Booking...
selecting the date 12...
Desk number 326A is found. Booking...
selecting the date 13...
Desk number 326A is found. Booking...
selecting the date 14...
Desk number 326A is found. Booking...
selecting the date 15...
Desk number 326A is found. Booking...
selecting the date 16...
Desk number 326A is found. Booking...
selecting the date 17...
Desk number 326A is found. Booking...
selecting the date 18...
Desk number 326A is found. Booking...
selecting the date 19...
Desk number 326A is found. Booking...
selecting the date 20...
Desk number 326A is found. Booking...
selecting the date 21...
Desk number 326A is found. Booking...
selecting the date 22...
Desk number 326A is found. Booking...
selecting the date 23...
Desk number 326A is found. Booking...


In [153]:
### Option 2: not use cookies

# username = "meysam.aghighi@ericsson.com"
# password = "myXtremelyStrongPassword"

# # Wait to find and click on "Sign in with Microsoft" 
# button = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CLASS_NAME, "sc-imWYAI"))
# )
# button.click()

# # Wait until the email input field is present and fill it in
# email_input = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "i0116"))
# )
# email_input.send_keys(username)  # Replace with the actual email

# # Click "Next"
# driver.find_element(By.ID, "idSIButton9").click()

# # Wait to find and click "Other ways to sign in"
# button = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "idA_PWD_SwitchToCredPicker"))
# )
# button.click()

# # Wait to find and click "use password"
# button = WebDriverWait(driver, 10).until(
#     EC.element_to_be_clickable((By.XPATH, "//div[normalize-space(text())='Use my password']"))
# )
# button.click()

# # Wait to find and fill in "password"
# password_input = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "i0118"))
# )
# password_input.send_keys(password)

# # Wait to find and click on "Sign in"
# button = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.ID, "idSIButton9"))
# )
# button.click()